In [2]:
import torch

In [3]:
with open("data/input.txt", "r") as f:
  text = f.read()

vocab_list = ''.join(sorted(list(set(text))))
vocab_size = len(vocab_list)

ctoi = { val: i for i, val in enumerate(vocab_list)}
itoc = { i: val for i, val in enumerate(vocab_list)}
encoder = lambda s: [ctoi[i] for i in s]
decoder = lambda enc: ''.join([itoc[i] for i in enc])

In [4]:
enc = encoder(text)
tt_split = 0.9

test = torch.tensor(enc[int(tt_split*len(enc)):])
train = torch.tensor(enc[:int(tt_split*len(enc))])

In [5]:
embd_size = 384
batch_size = 64
context_size = 256
dropout_thres = 0.2
n_heads = 6
n_layers = 6
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(1337)

In [6]:
class SelfAttentionHead(torch.nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.head_size = head_size
    self.key = torch.nn.Linear(embd_size, head_size, bias=False)
    self.value = torch.nn.Linear(embd_size, head_size, bias=False)
    self.query = torch.nn.Linear(embd_size, head_size, bias=False)

    self.drop = torch.nn.Dropout(dropout_thres)
    self.register_buffer("tril", torch.tril(torch.ones((context_size, context_size)))) 
  
  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x) # (B, T, head_size)
    v = self.value(x) # (B, T, head_size)
    q = self.query(x) # (B, T, head_size)
    
    W = q @ k.transpose(-1, -2) * self.head_size**(-0.5) # (B, T, T)
    W.masked_fill_(self.tril[:T, :T] == 0, float("-inf"))
    w_mask = torch.nn.functional.softmax(W, dim=-1) # (B, T, T)
    w_mask = self.drop(w_mask)

    return w_mask @ v # (B, T, head_size)

In [7]:
class MultiAttentionHead(torch.nn.Module):
  def __init__(self, n_heads, head_size):
    super().__init__()
    self.heads = torch.nn.ModuleList(
        SelfAttentionHead(head_size) for _ in range(n_heads)
    )
    self.proj = torch.nn.Linear(n_heads * head_size, embd_size)
    self.drop = torch.nn.Dropout(dropout_thres)
  
  def forward(self, x):
    st = torch.cat([head(x) for head in self.heads], dim=-1)
    return self.drop(self.proj(st))

In [8]:
class FeedForwardLayer(torch.nn.Module):
  def __init__(self, nin, nout):
    super().__init__()
    self.layers = torch.nn.Sequential(
        torch.nn.Linear(nin, 4*nout),
        torch.nn.ReLU(),
        torch.nn.Linear(4*nout, nout),
        torch.nn.Dropout(dropout_thres)
    )
  
  def forward(self, x):
    return self.layers(x)


In [9]:
class Block(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.sa_heads = MultiAttentionHead(n_heads, embd_size // n_heads)
    self.ffwd = FeedForwardLayer(embd_size, embd_size)

    self.ln1 = torch.nn.LayerNorm((embd_size,))
    self.ln2 = torch.nn.LayerNorm((embd_size,))
  
  def forward(self, x): 
    '''
    in: 
    - x: tensor (batch_size, context_size, embd_size)
    out: 
    - out: tensor (batch_size, context_size, embd_size)
    '''
    x = self.sa_heads(self.ln1(x)) + x # residual connections
    x = self.ffwd(self.ln2(x)) + x
    return x 

In [10]:
class BigramLanguageModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.feat_encoding = torch.nn.Embedding(vocab_size, embd_size)
    self.pos_encoding = torch.nn.Embedding(context_size, embd_size)

    self.blocks = torch.nn.Sequential(
        *[Block() for _ in range(n_layers)],
        torch.nn.LayerNorm((embd_size,)),
        torch.nn.Linear(embd_size, vocab_size)
    )

  def forward(self, xs, ys=None): # outputs logits, loss
    '''
    in:
    - xs: tensor (batch_size, context_size)
    - ys: tensor (batch_size, context_size) or None
    out:
    - logits: tensor (batch_size, context_size, vocab_size)
    - loss: tensor (1,) or None
    '''
    
    f = self.feat_encoding(xs) # (batch_size, context_size, embd_size)
    p = self.pos_encoding(torch.arange(0, xs.shape[1], device=device)) # (context_size, embd_size)

    x = f + p 
    logits = self.blocks(x)
    B, C, V = logits.shape

    if ys is not None:
      logits_ce = logits.reshape(B*C, V)
      ys_ce = ys.reshape(B*C)

      loss = torch.nn.functional.cross_entropy(logits_ce, ys_ce)
    else:
      loss = None
    return logits, loss

  def generator(self, max_length, batch_size_):
    '''
    in:
    - max_length: int
    out:
    - out: tensor (batch_size, max_length)
    '''
    out = torch.zeros((batch_size_, 1), dtype=torch.long, device=device)

    for _ in range(max_length - 1):
      last_char = out[:, -context_size:]

      logits, _ = self.forward(last_char)

      logits = logits[:, -1, :]
      probs = torch.nn.functional.softmax(logits, dim=-1)
      ntoken = torch.multinomial(probs, 1) 
      out = torch.cat((out, ntoken), dim=1)
    return out

In [11]:
class Trainer: 
    def __init__(self, model, train_data, optimizer, gpu_id, save_every): 
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.optimizer = optimizer
        self.model = DDP(self.model, device_ids=[self.gpu_id])
        self.save_every = save_every


    def _run_epoch(self, epoch): 
        self.train_data.sampler.set_epoch(epoch)

        x, y = get_batch(self.train_data)
        x, y = x.to(self.gpu_id), y.to(self.gpu_id)
        logits, loss = self.model(x, y)
        
        optimizer.zero_grad()
        loss.backward()
        
        with torch.no_grad():
            optimizer.step()

    def _save_checkpoint(self, epoch):
        ckp = self.model.module.state_dict()
        PATH = "checkpoint.pt"
        torch.save(ckp, PATH)
        print(f"Epoch {epoch} | Training checkpoint saved at {PATH}")


    def train(self, max_epochs):
        for epoch in tqdm(range(max_epochs)):
            self._run_epoch(epoch)
            if self.gpu_id == 0 and epoch % self.save_every == 0:
                self._save_checkpoint(epoch)


In [91]:
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import os

def ddp_setup(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12345"
    init_process_group(backend="nccl", rank=rank, world_size=world_size)

from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

def loader_setup():
    starts = torch.arange(0, len(train) - context_size - 1) 
    sampler = DistributedSampler(starts, shuffle=True) 
    def collate(starts_batch):
        xb = torch.stack([train[i:i+context_size] for i in starts_batch])
        yb = torch.stack([train[i+1:i+1+context_size] for i in starts_batch])
        return xb, yb
    
    loader = DataLoader(starts, batch_size=batch_size, sampler=sampler, collate_fn=collate, drop_last=True)
    return loader

In [92]:
epochs = 10000 
world_size = 4
save_every = 1000

def main(rank, world_size):
    model = BigramLanguageModel()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    loader = loader_setup()
    ddp_setup(rank, world_size)
    
    t = Trainer(model, loader, optimizer, rank, loader, save_every)
    t.train(epochs)
    
    destroy_process_group()

mp.spawn(main, args=(world_size,), nprocs=world_size)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/nlin06/.conda/envs/base.3.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nlin06/.conda/envs/base.3.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/nlin06/.conda/envs/base.3.11/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nlin06/.conda/envs/base.3.11/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ProcessExitedException: process 2 terminated with exit code 1

In [41]:
print(decoder(t.model.generator(1000, 1).tolist()[0]))


rdu oue, ocJ! o Doru ahastbbosejin i tUaB Oi hX E sps otn oaouK n obLmn doa
g:yc3eWCg HFu QNuesele sho
 aWv biqto oolhChn$TohFxmtGl haroaseaer:h  :OazoEu niVy;agSwroisdeoyGhntl-yrhs srecrgdJeelaOn;kR
elFpine
T!i pbyo:D thnbhe Mr my Euci
Hd tizimheb ayWarreedadne G,e qlbdrewaon ja  ly a tlcCWt ,i?f
 
oftwuysthOroQzK
t m r'l, p,
G
un
Koy er :zoI  n tt threUau on
, an to srlCQ tnCdoioQo
 rrtaoslnoevRninite snlvqtheurier3t bu s'slEGawt3EK d s nlrereo t une nd  sqmros lahTs aAwmo niqXlr:y a,d a w,cKmupIkIyi insge;
eg   tsaj'arr Etu t c m
U srs
ndilsoQatsqsp!r  smun
g 
n$ em jtQ'irssPf, mtnbnnt meus tie  Iht so!  Usgslr fethzvsQ,ftii snvdroniViXt
k
!IeNNd te
Rt iFe nvtt
itMuy hNfkK keos;innow  syet,
 leersiIs  tn Rts ZCon
li e
er VCn

ZoeQp
T g cseiF

 ro hell:dvufI s Lh!ey ar
e:
o sn dhenohn;Pe VNd o
oth'dir nBuJus
RdZ ta3iwiemy s t! ree rtO3Gn a rheou O: rwe rqore jk
a!Qo.r
eo, v gur
ey,n:
ll s svcw'lSwltordBlzu,IcadowVI le rhylqlcarai'

d
ve?oiullt !.wer tk so .eit
ssGe svool  s nt f ct,

In [ ]:
torch.save(model.state_dict(), "ckpt/model.0.pt")




In [16]:
model = BigramLanguageModel()

state = torch.load("checkpoint.pt", weights_only=True)
model.load_state_dict(state)
model = model.to(device)

In [18]:
print(decoder(model.generator(10000, 1).tolist()[0]))


Unbuchild severe;
The place is so next deliverance:
Speak an hollowly pray's moody,
Is crook'd is comentry, graceal sheltere,
Whether do pluck to me, that dies and merril,
Bear an art weeping in objoc, wherein I present
To advent more cause than great Bolingbroke?

JULIET:
The love bends upon a coddest
In old with honour'd of conting: so he the
chancest, homething but the recomplished action
yeyeth arrivaller.

Second Senator:
As throw's some comes of pardon.
Were your chiefes bear with rutures, with us!

FLORIZEL:
I removed every them, say, and not,
And they are but by their fellow. Here's as the vasleys:
I am not I say, I pray; the most all a loss
That they say 'going home will be answe,
You are deserved all gentle-pies. If you born
The shoulderable tranion, which you hope yet well me,
And crave in the tribul fruit-and gallant-s ebunch,
And princely must repost-bed,
When I come: to save your heads dames for't,
Your friar, and make friends thence, warm;
Was not your great hath I had 

In [ ]:
'''

Unbuchild severe;
The place is so next deliverance:
Speak an hollowly pray's moody,
Is crook'd is comentry, graceal sheltere,
Whether do pluck to me, that dies and merril,
Bear an art weeping in objoc, wherein I present
To advent more cause than great Bolingbroke?

JULIET:
The love bends upon a coddest
In old with honour'd of conting: so he the
chancest, homething but the recomplished action
yeyeth arrivaller.

Second Senator:
As throw's some comes of pardon.
Were your chiefes bear with rutures, with us!

FLORIZEL:
I removed every them, say, and not,
And they are but by their fellow. Here's as the vasleys:
I am not I say, I pray; the most all a loss
That they say 'going home will be answe,
You are deserved all gentle-pies. If you born
The shoulderable tranion, which you hope yet well me,
And crave in the tribul fruit-and gallant-s ebunch,
And princely must repost-bed,
When I come: to save your heads dames for't,
Your friar, and make friends thence, warm;
Was not your great hath I had some before
The gracies.

First Murderer:
But how sanctuard who he in, dost struct him?

First Murderer:
Up, nor sweet Oxford; for he comes.

First Soldier:
No;
For I must the hands of joyful idle find to his
comple, imal accupes me have been as ever:
He county and stays.

First Murderer:
if I must, as appeall to the foot Bianca!

CLARENCE:
But he, thinks you?

CLARENCE:
I may not have beed mine: I do of at
only honestable betrue you what hath a news him.

FLORIZEL:
No, Harry:
Fie, first,--

MENENIUS:
I we was awake information with of mine.
To this, worthy and only favour, sir?

Second Servant:
Well, then, 'tis age indecations show the lining
Against ic as so, they threatens are flected
With through and their hours: they do not bed awarry
Their life in no blessings o' the swills.

Third Gentleman:
Let's be old faith, some supposition, leave that,
As is a repentive it, or this bear callectors;
We shall be so better, dull aunt die now.

KING EDWARD IV:

LADY GREY:
Reasons, inducy, and go along at with be.

GLOUCESTER:
Thou hast spoken with thy love; and all poseen,
That with staying irons true foes before my head
And would plaging their powerful cunning doing,
Either marriage thee taps, though blacks up,
With paltiers, and traitorly Rivens, and nobles:
Thy having dares shed thou live to-morrow thee,
To every late thy headies that safet?
For we's it thy mind ten times, little one store,
What top? O, rugs; be men, see, thou hate, are desire.

nurse:
Anon, hark, Plantagenet and beyonded, and
fell to taste, who all coing, John of Gaunt, his grucious healt!

EXETER:
And Edward! You have not murd; we had not so brought
I bade before 't quick with prestitors; I hear would
lie see will seek the love ill sorrow for milly.

EDWARD:
Clifford, should you be not blood,--

NORD:
Lord Wilcome, Richard thou canst do to't, and not can?
I touch trial more dry. Come, fetch heart, who knows:
Romeo, art rusting his oath answer.
Thus! there is unat your plain, go not sit
Your brother's parcell; alas, is't your rise.
Down lads! Your grace manachether news!
My brother, that of hand with you bless coate.

DUCHESS OF YORK:
Ay, but then I'll die.

Lord:
I would act be clotted all at dining: your father's ambest,
For what till you have good on the wolf,
One and us wife sin in the saffects of ost!
Chy tolly a friar, and madam come by whoses:
Task it what tedic, you shall be before them?

First Huntsman:
Good marry, sir, both.

BARNA:
Fair Rome, madam, what dost thou art son?

Second Gentleman:
What answer there?

Messenger:
Ay, but when may do I be love beguiled.

GLOUCESTER:
Why now I will to tell I do't, Master Aufidius;
And be are thy unwilling is displeased.

BUCKINGHAM:
It were, that haste all better joy
To play a wholesomething thus I can do
Have dead to my perily. You, live you, good leave
That look'd to be custom your house: sue, get you cust,
I say gone; if that honour an ear,
The means which more destructing as I can intent,
But I'll not speak to mock you: I'll bite so.

ROMEO:
No man Kind of Rome, my lord, I know;
But not and wrong 'twill true the lord of Han,
Keep in vantage counted wasp, stay with blood
Where you'll scorns, Camillo, let 'em about the brace.

MENENIUS:
Your deserving rich work,
And yet shun him therein unto sea, scarr'd him.

CORIOLANUS:
O, sir!
True.
Intenting their noble traitors and more lies much
To sudden. But, let them do your service
Become game it to him done. The dukes to turn you
Which insurfeit they which should he die it:
'Tis like me wormwood for talk, my son,
All deaths on sleep'd, were that espectacles,
See, was puple's earth, or with the queer,
Showing the news, I then go the ead, let them them.

LADY ANNE:
How dost thou come? do not thee, God knows does from
thy conspiration piercing: and the bloody that
hanging the friend, for a happiness and suspicion
begs an accompleted him of unjustly current, and,
as as robuned as to foral the heart, Queen
him, we set that other shall accels upon his tender
of his world--a-moth most keep him is no out his
child lose consulle; let him shound him starm'd to talk. Marcius
banish'd with the land, all we can, and buy fretcher whole
pattend man, he would have got dewn to spare thee.
Now; for her thou liest on out in what a
palace we give to more me for a merrow,
To lord and his woman's hard-foul mothque hence;
And haste his net still, being their sun
delay nothing miscrebroke with Tybalt's leisure,
unto the effect ride them; was not to be all,
Down if that forger little her lives, certain,
Durying their peacer
The nipes man gave us in his lance to his friends:
He must will come away me not, and look so--

QUEEN ELIZABETH:
To what honour nod is Prince of my suit;
Is there I say, I say not satisfy
All last, to Freter villains with my person,
The husband, bearing upon Warwick strange-hour,
Becomes his ory wounded friend our splines,
And larr'd all his queen: which besile were dew,
And said is he, and he is a good.
Strive Edward's dog but trew me through.
The people thirty is the mire:
The hander doth little deputy and love
The law not the foe
And her brother'd life and provert tear
The times of my love clour from me. Henry 'aints
With that the sense o' the bigg' the ear--well--pashing
peaces, let's quoth: it ever deed,
For every matter as ere he, but over-leads;
to quaste; and, I'll reburing all the subject
fe-lesser threoples only once money?

BRUTUS:
Cut more 'tis or money. That I would I woo,
'Tis thus, I hear you 'ldges. This, I had had said
Hark censured by my lord; I had effected thee.

CLARENCE:
Bish't thou the prince'st another standing: fall
was the gall'd. Is that keep'st this followers?
But, if accursed now with me, and follow'd down.
Speak the jewel, fearful arms mine authority
Juliety, but not upon 's up: throw proud fereof words
Three-pile-crosssed mutining winter'd for me!--
I spite ood--that access the ragge,
With unall blesh flex your breast time to pieces.
I have plosed with you
As vaidage sedly things to yourself in eards.

SICINIUS:
Nay, thy fool, as I say, have from heaven.

MENENIUS:
Howen great's best much ill-bed, consul, and brace the
image!

MENENIUS:
To true when I was most prtinent for wich
surpress, charge and the loss of your pears with and
very given aside secrection will
make you: desiring, mine are your shepherds, yours
get to combacrers. You're that speak no more
cenerary, make but you good to charith me. God you, prisoner, my
better.

Nurse:
Peacee: God there's no marve I aboute this letter: he has
the hath neglige is privy.

DUKE VINCENTIO:
Pardon me, sure; for I do the rage: these
the services I will what did.

CLUSIO:
He wants
One to her vows; the digod, here in this offerion
Is here come, in the apprehension of her moiety:
'Zound abided,' and faithfully; and gentle ace
As we are in papellar to give thy love
Shall seeme which harms thy feign so love fasts
As a truer felt a cuffer'd from as offence.
In most whole-graced induction man!
What! had thou no sometimes? wilt thou ne'er first
He doth wrung in his arrival; which
The glory shall all the world as vest,
He writing to my old forgive.

First Lord:
Look not offer'd?

RIVERS:

HASTINGS:
Nor I, good for Claudio,
York I say my liege: it you commends him, sir,
Al, I must not belong the sun George Stanley, gave
me to, farewell mean to pright me; you think wherein?

SICINIUS:
What, man we will not inso?

BRUTUS:
To bear them, for these are were forements.

SICINIUS:
But as they are you.

MENENIUS:
Are new a we; you cannot well in madvised
Have you well at their couts do it.

First Senator:
He hath go with him would bear it, us.

BRUTUS:
He's much pold.

SICINIUS:
Not he would have you because
That, by young Rome: lesces night the seals
Did confinger him his death truth night to mar?
But thou speed the foolish in a purpose;
And wive, be solernel so left too!
Condue be mother'd to lose the state;
This she will go withal; conce where come thy letter.
Richard here! thou loft all York, and lose upon me;
And thou'lt stay with God shriek delay's thee.

Servant:
These I may horse, help to me on him, who lived.
I have despair'd to joy and it by'd.

PARINCE:
Her children, my lord, indums doth.

LADY GREY:
No, by send thou that fair voices with him.

ROMEO:
How she drums
The gracious lords, will I proud, renown'd
Of what seen uncle closeth at Odds,
By breedoms, my mount have I inforced with shore,
And beat our grave means too: be he but,
One back found me for to the house, to get him:
This dead perfect rights o' the complain.

DUKE OF AUMERLE:
Men, he comes she loves in the winter.

HENRY MONE:
Heart's name? my lords, 'I thank some come:
For what I do; more whether than I will;
Give you the castless with ocean with his son
When young could great for the Rome than in his
Measuure we met; and what they shall we
Thou that, without Augril of the house, when jest,
This wild that he we came from ourselves by ought
In all present live.

ARCHIDAMUS:
Well, wherein?

CAMILLO:
'There lay a bloody of Claudio: call'd mut on'
'''